In [1]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import os
from datetime import datetime

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] ='''--packages io.delta:delta-core_2.12:2.4.0 pyspark-shell'''

In [3]:
spark_session = SparkSession.builder \
            .appName("read_hdfs") \
            .master("local") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()
spark_session.conf.set("spark.sql.session.timeZone", "UTC+7")

23/12/15 12:46:18 WARN Utils: Your hostname, pm-GL65-Leopard-10SCXK resolves to a loopback address: 127.0.1.1; using 192.168.103.92 instead (on interface wlo1)
23/12/15 12:46:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pm/Downloads/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pm/.ivy2/cache
The jars for the packages stored in: /home/pm/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-28567e07-90fd-4518-abce-bd4ec481b88d;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 180ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |

In [8]:
df = spark_session.read.load("hdfs://localhost:9000/delta/stock2/symbol=A32/year=2023/month=12/day=15")
df.show()

+-------------+--------------------+---------+----------+-----------+--------+---------+--------+---------+--------+---------+----------+----------+---------+---------------+----------------------+--------+---------+--------+---------+--------+---------+----------+------+---------+---------+-----------+------------+-----------+
|exchange_name|          trade_time|ref_price|ceil_price|floor_price|b_qtty_3|b_price_3|b_qtty_2|b_price_2|b_qtty_1|b_price_1|pct_change|last_price|last_qtty|traded_quantity|total_value_traded_qtt|s_qtty_3|s_price_3|s_qtty_2|s_price_2|s_qtty_1|s_price_1|open_price|mid_px|max_price|min_price|foreign_buy|foreign_sell|close_price|
+-------------+--------------------+---------+----------+-----------+--------+---------+--------+---------+--------+---------+----------+----------+---------+---------------+----------------------+--------+---------+--------+---------+--------+---------+----------+------+---------+---------+-----------+------------+-----------+
|         

In [8]:
spark_session.stop()